In [ ]:
#Instalação das bibliotecas necessárias
!pip install yfinance #Biblioteca para coletar dados do Yahoo Finance

In [ ]:
# Importação das bibliotecas necessárias
import json
import os
from datetime import datetime

import yfinance as yf

In [ ]:
# Função para coletar os dados do Yahoo Finance
def fectch_stock_history_price(ticket):
    stock = yf.download(ticket, start="2020-01-01", end="2020-12-31")
    return stock